### Modelling to classify churn

In [51]:
import pandas as pd

dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


# Male/Female
le_X = LabelEncoder()
X[:, 2] = le_X.fit_transform(X[:, 2])

# Country column
ct = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)



X = X[:,1:]


# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [58]:
# Fitting classifier to the Training set
# Create your classifier here
import keras

#sequential model to initialize the neural networks and dense module to build the layers for NN
from keras.models import Sequential
from keras.layers import Dense

"""
# two ways to build a deep learning model
# 1. defining sequence of layers for NN
# 2. Or Defining the layers in the form of Graph
"""

#initializing the ANN
cls = Sequential()

"""
# adding the first layer, and input layers also our first hidden layer
# output_dim - number of nodes in the layer, can be average of input and output dim
# init - weigts to be initilized
# activation - activtion function you want to use in the layer
"""

# adding input and first hidden layer
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu',input_dim = 11))

# adding second hidden layer
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu'))

# adding third hidden layer
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu'))


# adding output layer
"""
if we have more than one dependent variables, 
whihc might be the case when we do one hot encoding for multi class dependent variable.
We should pick the out_dim based on number of varibales we have and our activation will be 'softmax'
"""
cls.add(Dense(output_dim = 1,init = 'uniform',activation = 'sigmoid'))


# compiling the ANN
"""
optimizer = algorithm to optimize for the weights (GD,SGD,MGD)
            adam is one of the SGD available
loss = algorithm used to calculate the cost or loss between the actual vs predicted
        log loss is used for binary classification (binary_crossentropy for binary)
metrics = metric used to evaluate your model
"""
cls.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

# fitting the ANN on our training data
"""
X - your training independent varibales data
y - your training dependent varibal data
batch_size - decides whether we are doing the GD, SD or MGD
nb_epochs - number of times the 
"""
cls.fit(X_train,y_train,batch_size = 10,nb_epoch = 100)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
7500/7500 [==============================] - 1s 128us/step - loss: 0.4887 - accuracy: 0.7956
Epoch 2/100
7500/7500 [==============================] - 1s 87us/step - loss: 0.4260 - accuracy: 0.7981
Epoch 3/100
7500/7500 [==============================] - 1s 86us/step - loss: 0.4156 - accuracy: 0.8264
Epoch 4/100
7500/7500 [==============================] - 1s 83us/step - loss: 0.4095 - accuracy: 0.8297
Epoch 5/100
7500/7500 [==============================] - 1s 82us/step - loss: 0.4049 - accuracy: 0.8345
Epoch 6/100
7500/7500 [==============================] - 1s 83us/step - loss: 0.4040 - accuracy: 0.8345
Epoch 7/100
7500/7500 [==============================] - 1s 83us/step - loss: 0.4022 - accuracy: 0.8352
Epoch 8/100
7500/7500 [==============================] - 1s 83us/step - loss: 0.4017 - accuracy: 0.8340
Epoch 9/100
7500/7500 [==============================] - 1s 83us/step - loss: 0.4006 - accuracy: 0.8353
Epoch 10/100
7500/7500 [==============================] - 1s 83

In [93]:
from sklearn.metrics import confusion_matrix


def get_confMatrix(y_pred,y_test,threshold):
    y_pred = [1 if y >=threshold else 0 for y in y_pred] 
    # Making the Confusion Matrix
    return confusion_matrix(y_test, y_pred)

# Predicting the Test set results
y_pred = cls.predict(X_test)

cm = get_confMatrix(y_pred,y_test,0.5)

print("accuracy is {}".format((cm[0,0]+cm[1,1])/len(y_test)))
print("precision is {}".format((cm[1,1])/(cm[1,1]+cm[0,1])))
print("recall is {}".format((cm[1,1])/(cm[1,0]+cm[1,1])))




import numpy as np
np.random.seed(777)
TPR = []
FPR = []
for threshold in np.linspace(1,0,100):
    cm = get_confMatrix(y_pred,y_test,threshold)
    TPR.append(cm[1,1]/(len(y_test)))
    FPR.append(cm[0,0]/(len(y_test)))
    
    
import plotly.offline as pyo
import plotly.graph_objs as go


fig = go.Figure(data = [go.Scatter(x = FPR,
                                   y = TPR,
                                   mode = 'lines')],
                layout = go.Layout(title = 'AUC-ROC',
                                   xaxis = {'title':'FPR'},
                                   yaxis = {'title':'TPR'}))

pyo.plot(fig)

accuracy is 0.8608
precision is 0.7158176943699732
recall is 0.5245579567779961


'temp-plot.html'

In [103]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
singlePredict = cls.predict(sc.transform(np.array([[0.0, 0.0, 800, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
singlePredict = (new_prediction > 0.5)
print(singlePredict)

[[False]]
